In [1]:
from ls_helper.new_models import get_p_access, platforms_overview2, ProjectData
from ls_helper.settings import SETTINGS

twitter = "twitter"
en = "en"
tw_en = {"platform": twitter, "language": en}
_default = tw_en

po: ProjectData = platforms_overview2.get_project(get_p_access(**_default))

In [2]:
po.get_agreement_data()